In [1]:
import os
from PIL import Image

# Paths
root_dir = "PotatoData/train"   # replace with your dataset path
img_size = (224, 224)  # YOLO cls default

def resize_images(folder):
    for subdir, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(subdir, file)
                try:
                    img = Image.open(img_path).convert("RGB")
                    img = img.resize(img_size)
                    img.save(img_path)  # overwrite
                except Exception as e:
                    print(f"Error resizing {img_path}: {e}")

# Apply to all sets
for split in ["train", "valid", "test"]:
    resize_images(os.path.join(root_dir, split))

print("✅ All images resized successfully!")

✅ All images resized successfully!


In [2]:
from ultralytics import YOLO

# load classification model (ResNet-like backbone)
model = YOLO("yolov8n-cls.pt")

# train
model.train(
    data="PotatoData/train" ,
    epochs=20,
    imgsz=224,
    batch=16
)


Ultralytics 8.3.201  Python-3.11.4 torch-2.8.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=PotatoData/train, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, p

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022EC2BB4690>
curves: []
curves_results: []
fitness: 0.9995274245738983
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9990548491477966, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9995274245738983}
save_dir: WindowsPath('D:/USER/OneDrive/Desktop/Final_yr/notebook/runs/classify/train5')
speed: {'preprocess': 0.0005253308208336674, 'inference': 4.68518908317987, 'loss': 3.317580844446425e-05, 'postprocess': 7.759926049853015e-05}
task: 'classify'
top1: 0.9990548491477966
top5: 1.0

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Load YOLO training logs
df = pd.read_csv("runs/classify/train5/results.csv")

# Show available columns
print(df.columns)



Index(['epoch', 'time', 'train/loss', 'metrics/accuracy_top1',
       'metrics/accuracy_top5', 'val/loss', 'lr/pg0', 'lr/pg1', 'lr/pg2'],
      dtype='object')


In [17]:
import matplotlib.pyplot as plt
import pandas as pd

# Load logs
df = pd.read_csv("runs/classify/train5/results.csv")

# 📉 Loss Curve
plt.figure(figsize=(10,5))
plt.plot(df["epoch"], df["train/loss"], label="Train Loss", marker="o")
plt.plot(df["epoch"], df["val/loss"], label="Validation Loss", marker="s")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.savefig("loss_curve.png")   # ✅ saves plot
plt.close()

# 📈 Accuracy Curve
plt.figure(figsize=(10,5))
plt.plot(df["epoch"], df["metrics/accuracy_top1"], label="Top-1 Accuracy", marker="o")
plt.plot(df["epoch"], df["metrics/accuracy_top5"], label="Top-5 Accuracy", marker="s")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Validation Accuracy")
plt.legend()
plt.grid(True)
plt.savefig("accuracy_curve.png")   # ✅ saves plot
plt.close()

print("✅ Plots saved as loss_curve.png and accuracy_curve.png")


✅ Plots saved as loss_curve.png and accuracy_curve.png


In [9]:
# Evaluate on validation set (default) or specify test set
metrics = model.val(data="PotatoData/test")

print(metrics)  # prints all metrics (top1_acc, top5_acc, precision, recall, etc.)


Ultralytics 8.3.201  Python-3.11.4 torch-2.8.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
WARNING Dataset 'split=train' not found at D:\USER\OneDrive\Desktop\Final_yr\notebook\PotatoData\test\train
Found 1807 images in subdirectories. Attempting to split...
Splitting D:\USER\OneDrive\Desktop\Final_yr\notebook\PotatoData\test (2 classes, 1807 images) into 80% train, 20% val...
Split complete in D:\USER\OneDrive\Desktop\Final_yr\notebook\PotatoData\test_split 
train: D:\USER\OneDrive\Desktop\Final_yr\notebook\PotatoData\test_split\train... found 1444 images in 2 classes  
val: D:\USER\OneDrive\Desktop\Final_yr\notebook\PotatoData\test_split\val... found 363 images in 2 classes  
test: None...
val: Fast image access  (ping: 0.10.0 ms, read: 8.41.4 MB/s, size: 65.3 KB)
val: Scanning D:\USER\OneDrive\Desktop\Final_yr\notebook\PotatoData\test_split\val... 363 images, 0 corrupt: 100% ━━━━━━━━━━━━ 363/3

In [10]:
# Get specific metrics
print(f"Top-1 Accuracy: {metrics.top1:.2f}")
print(f"Top-5 Accuracy: {metrics.top5:.2f}")


Top-1 Accuracy: 1.00
Top-5 Accuracy: 1.00


In [ ]:
# results = model.predict("PotatoData/test/Not_Potato/01482fdd-5dfd-4190-bbc0-6189a3161fae___JR_HL-7983_JPG.rf.9820ed2a5dbe13e17857622ed16e20e3.jpg")

results = model.predict("PotatoData/test/Potato/1106c3fc-92cb-41a6-a6c6-8f08b9b45108___RS_HL-1914_flipTB_JPG.rf.db639c064704912a6ba43bd8b23be1bf.jpg")

print(results)



image 1/1 d:\USER\OneDrive\Desktop\Final_yr\notebook\PotatoData\test\Potato\1106c3fc-92cb-41a6-a6c6-8f08b9b45108___RS_HL-1914_flipTB_JPG.rf.db639c064704912a6ba43bd8b23be1bf.jpg: 224x224 Potato 1.00, Not_Potato 0.00, 9.0ms
Speed: 5.1ms preprocess, 9.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
[ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'Not_Potato', 1: 'Potato'}
obb: None
orig_img: array([[[129, 119, 142],
        [126, 116, 139],
        [129, 119, 142],
        ...,
        [116, 109, 130],
        [112, 105, 126],
        [104,  97, 118]],

       [[125, 115, 138],
        [122, 112, 135],
        [126, 116, 139],
        ...,
        [116, 109, 130],
        [117, 110, 131],
        [113, 106, 127]],

       [[128, 118, 141],
        [126, 116, 139],
        [131, 121, 144],
        ...,
        [114, 107, 128],
        [116, 109, 130],
        [112, 105, 126]],

       ...,

       [[152, 